In [1]:
import json
import os
import numpy as np
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import soundfile as sf
import re
from multiprocess import Pool
import itertools

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [11]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-v2", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)
lang_model.predict('hello my name', k = 10)

fasttext.ftz:   0%|          | 0.00/227M [00:00<?, ?B/s]

(('__label__standard-english',
  '__label__local-english',
  '__label__standard-malay',
  '__label__socialmedia-indonesian',
  '__label__local-malay',
  '__label__other'),
 array([9.12180483e-01, 4.69220504e-02, 4.03920077e-02, 5.50693308e-04,
        1.30474637e-05, 1.07987826e-05]))

In [9]:
lang_model.predict('hello name saye', k = 10)

(('__label__local-malay',
  '__label__standard-malay',
  '__label__socialmedia-indonesian',
  '__label__other'),
 array([0.69633353, 0.29375371, 0.00906781, 0.00088501]))

In [3]:
def chunk(alignment, reject = -7, minimum_length = 1.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    temp[-1]['end'] = float(temp[-1]['end']) + 0.1
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            temp[-1]['end'] = float(temp[-1]['end']) + 0.1
            alls.append(temp)
    return alls

In [4]:
data = defaultdict(list)
with open('prepared-pseudolabel.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        l = json.loads(l)
        data[l['audio_filename']].append((no, l))
        
len(data)

3085595it [00:16, 185628.88it/s]


1961155

In [5]:
rows = list(data.values())
len(rows)

1961155

In [6]:
!rm -rf prepared-pseudolabel-chunks
!mkdir prepared-pseudolabel-chunks

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

def loop(data):
    data, _ = data
    new_data = []
    for d in tqdm(data):
        
        aligns, scores = [], []
        for i in d:
            f = f'prepared-pseudolabel_alignment/{i[0]}.alignment'
            try:
                with open(f) as fopen:
                    align = json.load(fopen)
                    score = np.sum([s['score'] for s in align])
                    aligns.append(align)
                    scores.append(score)
            except:
                aligns.append([])
                scores.append(-9999)

        argmax = np.argmax(scores)
        no = d[argmax][0]
        text = d[argmax][1]['new_text']
        
        rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
        if any([s == rt_ for s in rejected]):
            continue
            
        split = text.split()
        ones = [w for w in split if len(w) <= 1]
        if (len(ones) / len(split)) >= 0.5:
            continue
            
        if any([(len(set(w)) / len(w)) < 0.3 for w in split]):
            continue
        
        try:
            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
            repeat = (dense > 3).sum() >= 1
            if repeat:
                continue
        except:
            continue
            
        lang = text.split('<|startoftranscript|><|')[1].split('|')[0]
        cleaned_text = re.sub(r"<\|.*?\|>", "", text).strip()
        if lang_model.predict(cleaned_text)[0][0] == '__label__english':
            predict_lang = 'en'
        else:
            predict_lang = 'ms'
        
        chunks = chunk(aligns[argmax])
        audio_filename = d[argmax][1]['audio_filename']
        if len(chunks):
            y, sr = sf.read(audio_filename)
            for k, c in enumerate(chunks):
                
                y_ = y[int(sr * c[0]['start']): int(sr * c[-1]['end'])]
                
                skip = False
                
                for c_ in c:
                    if (c_['end'] - c_['start']) > 2:
                        skip = True
                        break
                if skip:
                    continue
                
                for no_ in range(len(c)):
                    if no_ > 0 and (c[no_]['start'] - c[no_ - 1]['end']) > 1.2:
                        skip = True
                        break
                if skip:
                    continue
                    
                ts = []
                
                min_t = min([c_['start'] for c_ in c])
                
                for c_ in c:
                    start = min(timestamps, key=lambda t: abs(t - (c_['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (c_['end'] - min_t)))
                    w = c_['text']
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                    
                new_f = os.path.join('prepared-pseudolabel-chunks', f'{no}-{k}.mp3')
                if not os.path.exists(new_f):
                    sf.write(new_f, y_, sr)
                    
                word = ''.join(ts)
                word = f"<|startoftranscript|><|{predict_lang}|><|transcribeprecise|>{word}<|endoftext|>"
                
                segments, temp = [], [c[0]]
                last_t = c[0]['end']
                for c_ in c[1:]:
                    if (c_['start'] - last_t) > 0.3:
                        segments.append(temp)
                        temp = []

                    last_t = c_['end']
                    temp.append(c_)

                if len(temp):
                    segments.append(temp)
                
                ts = []
                for s in segments:
                    start = min(timestamps, key=lambda t: abs(t - (s[0]['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (s[-1]['end'] - min_t)))
                    w = ' '.join([c_['text'] for c_ in s])
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                
                ts = ''.join(ts)
                new_text = f"<|startoftranscript|><|{predict_lang}|><|transcribe|>{ts}<|endoftext|>"
                
                new_data.append({
                    'audio_filename': new_f,
                    'word_timestamp': word,
                    'segment_timestamp': new_text,
                })
                
    return new_data

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
r = loop((rows[:10], 0))
len(r)

100%|███████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.52it/s]


45

In [10]:
r[-6]

{'audio_filename': 'prepared-pseudolabel-chunks/15-3.mp3',
 'word_timestamp': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> ada<|0.08|><|0.14|> kefew<|0.30|><|0.40|> juga<|0.56|><|0.78|> zaman?<|0.98|><|1.02|> Ada.<|1.12|><|1.50|> Masih<|1.76|><|1.82|> ada?<|2.04|><|endoftext|>',
 'segment_timestamp': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> ada kefew juga zaman? Ada.<|1.12|><|1.50|> Masih ada?<|2.04|><|endoftext|>'}

In [11]:
import IPython.display as ipd
ipd.Audio(r[-6]['audio_filename'])

In [12]:
r = multiprocessing(rows, loop, cores = 50)

 35%|████████████████████████████▏                                                    | 13675/39223 [20:42<35:52, 11.87it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
len(r) / 1e6

5.862274

In [16]:
len(r)

5862274

In [17]:
!du -hs prepared-pseudolabel-chunks

130G	prepared-pseudolabel-chunks


In [15]:
import IPython.display as ipd
ipd.Audio(r[3000000]['audio_filename'])

In [20]:
r[3000000]

{'audio_filename': 'prepared-pseudolabel-chunks/1577939-3.mp3',
 'word_timestamp': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> sebelum<|0.24|><|0.32|> ni.<|0.36|><|0.66|> Dan<|0.78|><|0.86|> kita<|0.98|><|1.04|> orang<|1.28|><|1.44|> menang<|1.76|><|2.18|> kat<|2.26|><|2.30|> sana<|2.54|><|endoftext|>',
 'segment_timestamp': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> sebelum ni.<|0.36|><|0.66|> Dan kita orang menang<|1.76|><|2.18|> kat sana<|2.54|><|endoftext|>'}

In [21]:
import pandas as pd

pd.DataFrame(r).to_parquet('pseudolabel-whisper-word-timestamp-chunk.parquet')

In [22]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="pseudolabel-whisper-word-timestamp-chunk.parquet",
    path_in_repo="data/malaysian_context_v2-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-STT-Whisper",
    repo_type="dataset",
)

Uploading...:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/commit/5f31ce6f101899892e05f79a36ceef20a42ebabc', commit_message='Upload data/malaysian_context_v2-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='5f31ce6f101899892e05f79a36ceef20a42ebabc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-STT-Whisper'), pr_revision=None, pr_num=None)

In [23]:
from glob import glob
import os

repository = 'mesolitica/Malaysian-STT-Whisper'
folder = 'prepared-pseudolabel-chunks'
files = glob(f'{folder}/*.mp3')
len(files)

5862274

In [25]:
import zipfile
import time
from huggingface_hub import HfFileSystem
from huggingface_hub import HfApi
from tqdm import tqdm

partition_size = 5e+9

In [26]:
def loop(files):
    files, index = files
    current_index = 0
    api = HfApi()
    fs = HfFileSystem()
    total = 0
    temp = []
    for i in tqdm(range(len(files))):
        s = os.stat(files[i]).st_size
        if s + total >= partition_size:
            part_name = f"{folder}-{index}-{current_index}.zip"
                
            with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for f in temp:
                    zipf.write(f, arcname=f)

            while True:
                try:
                    os.system(f'huggingface-cli upload {repository} {part_name} --repo-type=dataset')
                    break
                except:
                    time.sleep(60)

            os.remove(part_name)
            
            current_index += 1
            temp = [files[i]]
            total = s
        else:
            temp.append(files[i])
            total += s
        
    if len(temp):
        part_name = f"{folder}-{index}-{current_index}.zip"

        with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for f in temp:
                zipf.write(f, arcname=f)

        while True:
            try:
                os.system(f'huggingface-cli upload {repository} {part_name} --repo-type=dataset')
                break
            except:
                time.sleep(60)

        os.remove(part_name)

In [28]:
multiprocessing(files, loop, cores = 10, returned = False)

 37%|███████████████████████████▊                                               | 217567/586227 [00:00<00:00, 449994.75it/s]Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..


https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-5-0.zip


 77%|███████████████████████████████████████████████████████████                  | 450024/586227 [09:22<02:02, 1115.47it/s]

https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-0-0.zip


 71%|██████████████████████████████████████████████████████▊                      | 417758/586227 [09:30<02:37, 1072.31it/s]

...

https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-2-1.zip


100%|██████████████████████████████████████████████████████████████████████████████| 586227/586227 [19:10<00:00, 509.60it/s]
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..
Uploading files using Xet Storage..


https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-0-2.zip


100%|██████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 64280.52it/s]
Uploading files using Xet Storage..


https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-10-0.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-9-2.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-6-2.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-5-2.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-2-2.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-7-2.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-4-2.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-3-2.zip
https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/blob/main/prepared-pseudolabel-chunks-1-2.zip
